In [ ]:
# import sys
# sys.path.append('..')

# Data Extraction/Scrapping

Note: This script will not give the all 10-K FORM of all 2568 URLs at a single shot. Give only 1000 URLs at time.

In [ ]:
import re
import time
import requests
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import urllib.request as url
from bs4 import BeautifulSoup as bs
import pandas as pd

In [ ]:
%%time
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
phd_data = pd.read_csv("../csv/10k_filing_info_train_links-1570100710859.csv")

In [ ]:
phd_data.head()

In [ ]:
ticker_counts = phd_data['ticker'].value_counts()
ticker_counts_filtered = ticker_counts[:1]
# ticker_counts_filtered = ticker_counts[ticker_counts > 20]
ticker_counts_filtered

In [ ]:
# tmk_data = phd_data[phd_data['ticker'] == 'TMK']
# tmk_data.sort_values(by='filing_date')

In [ ]:
all_link = phd_data['10k_link']
all_cik = phd_data['cik']
all_ticker = phd_data['ticker']
all_filing_date = phd_data["filing_date"]
all_long_term_outlook = phd_data["long_term_outlook"]

all_data  = list(zip(all_ticker,all_cik,all_filing_date,all_long_term_outlook,all_link))

all_data = [
    (ticker, cik, filing_date, long_term_outlook, link)
    for ticker, cik, filing_date, long_term_outlook, link in all_data
    if ticker in ticker_counts_filtered.index
]

In [ ]:
len(all_data)

In [ ]:
all_data[0]

In [ ]:
#type(all_link)

In [ ]:
URL = "https://www.sec.gov/Archives/edgar/data/1031296/000103129612000012/0001031296-12-000012.txt"
#url = "https://www.sec.gov/Archives/edgar/data/1031296/000103129612000012/0001031296-12-000012.txt"

In [ ]:
URL = ["https://www.sec.gov/Archives/edgar/data/1031296/000103129612000012/0001031296-12-000012.txt","https://www.sec.gov/Archives/edgar/data/21665/000162828015000846/0001628280-15-000846.txt"]

In [ ]:
data_10k = {"ticker":None,"cik":None,"filing_date":None,"sentence":None,"long_term_outlook":None}

In [ ]:
data_10k

In [ ]:
def get_url_data(URL):
  try:
    print ("executing URL",URL[4])
    time.sleep(5)
    data = url.urlopen(URL[4]).read()
    print("data: ", data)
    soup = bs(data,'html.parser')
    print("soup: ", soup)
    doc = soup.find_all("type")
    doc = soup.find(name='body')
    print("found doc: ", doc) 
    return (doc,URL[0:4])
  except:
    print ("crashed for the URL",URL)
    #doc = soup.find(name='body')
    return (None,URL[0:4])

In [ ]:
def scrap_data(doc,xdata):
  main_data = []
  for i in doc:
    cont = i.contents
    if cont[0].strip() =="10-K":
      print ("Found")
      main_data.append(cont[1].text)
  return (main_data,xdata)


In [ ]:
#doc = get_url_data(URL)

In [ ]:
#d = scrap_data(doc)
#{'ticker': None, 'cik': None, 'filing_date': None, 'sentence': None}

In [ ]:
def write_data(d,x):
    if data_10k.get('sentence'):
        ts = data_10k.get('sentence')
        tk = data_10k.get('ticker')
        tcik = data_10k.get('cik')
        tdate = data_10k.get('filing_date')
        tt = data_10k.get('long_term_outlook')
    else:
        ts,tk,tcik,tdate,tt = [],[],[],[],[]
    ts.append(d)
    tk.append(x[0])
    tcik.append(x[1])
    tdate.append(x[2])
    tt.append(x[3])
    data_10k['sentence'] = ts.copy()
    data_10k['ticker'] = tk.copy()
    data_10k['cik'] = tcik.copy()
    data_10k['filing_date'] = tdate.copy()
    data_10k['long_term_outlook'] = tt.copy()
    print("Sucessfully Write!!..")

In [ ]:
# write_data(d)all_data

In [ ]:
all_data

In [ ]:
%%time

c=0
# for u in all_data[0:1000]:#instead of passing single URL passed 8
for u in all_data[:2]:
  print("Count:{0}".format(c))  
  doc, xdata = get_url_data(u)
  print("doc",doc)
  if doc:
    d,x = scrap_data(doc,xdata)
    #print(d)
    write_data(d,x)
  else:
    print("data crashed")
    #write_data("data crashed")
  c += 1

In [ ]:
#%%time
#for u in all_link[1:11]:#instead of passing single URL passed 8
#  doc = get_url_data(u)
#  if doc:
#    d = scrap_data(doc)
#    #print(d)
##    write_data(d)
#  else:
#    print("data crashed")
#    #write_data("data crashed")

In [ ]:
ar = pd.DataFrame(data_10k)

In [ ]:
ar.head(10)

In [ ]:
ar.to_csv("final_updated_1000.csv",index = False)